# Model Formulation Notebook

**ChatGPT Links**
- V1 - Formulation + Initial implementation (done by Wei Ting) : https://chatgpt.com/share/67fbd1dd-7690-8005-bddf-c4d48bf8f432
- V2 - Refinement of code + Prepare for integration in backend server (done by Wei Kang ) : https://chatgpt.com/share/67fbe679-a840-8013-805e-73739cef04a8
- V3 - Refinement of Model Constraints (done by Wei Kang) : https://chatgpt.com/share/6803ee83-a6f0-8013-a7b8-fb04444c537d

## Import

In [1]:
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime, timezone, timedelta
# replace with the actual filenames
_diets_df = pd.read_csv("../../data/final/processed_final_diets.csv")
_ex_df    = pd.read_csv("../../data/final/processed_final_exercises.csv")

print(f"Loaded {len(_diets_df)} diets and {len(_ex_df)} exercises")


Loaded 8293 diets and 93 exercises


## Helper functions to simulate filtering out of data and exercises as well as pre-computations


In [2]:
def filter_diets(user_params: dict) -> pd.DataFrame:
    df = _diets_df.copy()
    dr = [d.lower() for d in user_params.get("dietRestrictions", [])]
    if "none" not in dr:
        df = df[df["diet_type"].str.lower().isin(dr)]
    vp = [v.lower() for v in user_params.get("varietyPreferences", [])]
    if "none" not in vp:
        df = df[df["cuisine_type"].str.lower().isin(vp)]
    print(f"Filtered diets ({len(df)} rows)")
    return df

def filter_exercises(user_params: dict) -> pd.DataFrame:
    df = _ex_df.copy()
    fl = user_params.get("fitnessLevel", "").lower()
    if fl and fl != "none":
        df = df[df["difficulty_level"].str.lower() == fl]
    pl = user_params.get("preferredLocation", "").lower()
    if pl and pl != "none":
        df = df[df["workout_location"].str.lower() == pl]
    wt = user_params.get("preferredWorkoutType", "").lower()
    if wt and wt != "none":
        df = df[df["activity_type"].str.lower() == wt]
    print(f"Filtered exercises ({len(df)} rows)")
    return df

In [3]:
from math import ceil


def compute_user_metrics(user: dict) -> dict:
    """Calculate user metrics based on personal information."""
    # Constants
    CALORIES_PER_KG = 7700  # Approximate calories per kg of body weight
    ACTIVITY_MULTIPLIER = 1.2

    w, h, a = user["weight"], user["height"], user["age"]
    s = 5 if user["gender"].lower() == "male" else -161

    # 1. BMR
    bmr = 10 * w + 6.25 * h - 5 * a + s

    # 2. Activity assume SEDENTARY = 1.2 RATIO
    tdee = bmr * ACTIVITY_MULTIPLIER

    # 3. Weight change & daily calorie delta
    gw = user["goalWeight"]
    wt_change = gw - w
    today = datetime.now(timezone.utc)
    tgt   = datetime.fromisoformat(user["goalTargetDate"].replace("Z","+00:00"))
    delta = tgt - today
    days = max( ceil(delta.total_seconds() / 86400), 1 )
    cal_delta = CALORIES_PER_KG * wt_change / days

    # 4. Target calories per day
    target_cal_pd = tdee + cal_delta

    return {
        "BMR": round(bmr,2),
        "TDEE": round(tdee,2),
        "weight_change_kg": round(wt_change,2),
        "days_to_target": days,
        "calorie_change_per_day": round(cal_delta,2),
        "target_calorie_per_day": round(target_cal_pd,2),
    }

## Example request body from frontend

In [28]:
sample_user = {
    "age": 35,
    "daysWeek": 3,
    "dietRestrictions": ["none"],
    "fitnessLevel": "intermediate",
    "freeTime": 3,
    "gender": "male",
    "goalTargetDate": "2025-05-15T04:48:00.000Z",
    "goalType": "weight_loss",
    "goalWeight": 65,
    "height": 175,
    "mealPrepTime": 15,
    "mealsPerDay": 3,
    "name": "wei kang",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 70
}

## Prep work to calculate before feeding the details into Model

In [29]:
# filter data
diets_df = filter_diets(sample_user)
ex_df    = filter_exercises(sample_user)

# compute metrics
metrics = compute_user_metrics(sample_user)

# convert to lists of dicts for easy iteration in Gurobi
diet_options = diets_df.to_dict(orient="records")
ex_options   = ex_df.to_dict(orient="records")

print("User metrics:", metrics)
print(f"{len(diet_options)} diet options, {len(ex_options)} exercise options ready for modeling.")

Filtered diets (8293 rows)
Filtered exercises (57 rows)
User metrics: {'BMR': 1623.75, 'TDEE': 1948.5, 'weight_change_kg': -5, 'days_to_target': 26, 'calorie_change_per_day': -1480.77, 'target_calorie_per_day': 467.73}
8293 diet options, 57 exercise options ready for modeling.


In [30]:
print("Diet Dataset Preview:")
display(diets_df.head())
print(len(diets_df))
print("Exercise Dataset Preview:")
display(ex_df.head())
print(len(ex_df))

Diet Dataset Preview:


,recipe,diet_type,cuisine_type,total_time_in_minutes,calories,fat,carbs,protein
0,Chia And Blackberry Pudding,keto,kosher,45.0,437.0,38.0,8.0,8.0
1,Cinnamon Chiller,keto,south american,10.0,145.0,4.0,1.6,0.6
2,Cheesy Low-Carb Omelet,keto,chinese,10.0,451.0,36.0,3.0,33.0
3,Angel Eggs,keto,middle eastern,30.0,184.0,15.0,1.0,12.0
4,Roasted Onions And Green Beans,keto,italian,25.0,214.0,19.4,3.7,8.3


8293
Exercise Dataset Preview:


,exercise_name,calories_burned_30min,difficulty_level,calories_burned_per_min,workout_location,activity_type
0,Push-ups,200,Intermediate,6.666667,home,General
4,Mountain Climbers,240,Intermediate,8.000000,home,General
6,Bicycle Crunches,210,Intermediate,7.000000,home,General
7,Dips,180,Intermediate,6.000000,home,General
9,Russian Twists,190,Intermediate,6.333333,home,General


57


In [31]:
# More prep work
T = metrics["days_to_target"]
target_calorie_pd = metrics["target_calorie_per_day"]
goal = sample_user["goalType"].replace("_", " ")
# freeTime is in hours/day → convert to minutes/day
FT_day = sample_user["freeTime"] * 60
# mealPrepTimePerDay is total meal prep time (mins) per day
MPT_day = sample_user["mealPrepTime"]  * sample_user["mealsPerDay"]
# meals per day
M = sample_user["mealsPerDay"]
# desired workout days per week
W = sample_user["daysWeek"]

num_weeks = T // 7
has_partial_week = T % 7 > 0

print(f"target_calorie_pd = {target_calorie_pd}")
print(f"num_days:{T}")
print(f"num_weeks: {num_weeks}, has_partial_week: {has_partial_week}")
print(f"Goal: {goal}")
print(f"Free time per day: {FT_day} mins")
print(f"Total meal‐prep time per day: {MPT_day} mins")
print(f"Meals per day: {M}")
print(f"Workout days per week: {W}")

target_calorie_pd = 467.73
num_days:26
num_weeks: 3, has_partial_week: True
Goal: weight loss
Free time per day: 180 mins
Total meal‐prep time per day: 45 mins
Meals per day: 3
Workout days per week: 3


## Model formulation and actual run-time

In [32]:
from gurobipy import Model, GRB, quicksum
model = Model("FitPlanner")
model.Params.OutputFlag = 0 # Turn off the display
model.setParam('TimeLimit', 300)  # Set time limit to 5 minutes

# Sets
days = range(T)
meals = list(diets_df.index)  # i in I
exercises = list(ex_df.index)  # j in J

# Parameters (extract from dataset)`
recipe = {i: diets_df.loc[i, 'recipe'] for i in meals}
cal = {i: diets_df.loc[i, 'calories'] for i in meals}
fat = {i: diets_df.loc[i, 'fat'] for i in meals}
carb = {i: diets_df.loc[i, 'carbs'] for i in meals}
protein = {i: diets_df.loc[i, 'protein'] for i in meals}
prep_time = {i: diets_df.loc[i, 'total_time_in_minutes'] for i in meals}
ex_name = {j: ex_df.loc[j, 'exercise_name'] for j in exercises}
location = {j: ex_df.loc[j, 'workout_location'] for j in exercises}
burn_rate = {j: ex_df.loc[j, 'calories_burned_per_min'] for j in exercises}
workout_type = {j: ex_df.loc[j, 'activity_type'] for j in exercises}

# Max time by workout type
max_time = {j: 60 if workout_type[j] in {"Sports"} else
            30 if workout_type[j] in {"Outdoor/Water"} else 20 for j in exercises}

# Decision variables
x = model.addVars(meals, days,     vtype=GRB.BINARY,    name="x")
y = model.addVars(exercises, days, vtype=GRB.BINARY,    name="y")
t = model.addVars(exercises, days, vtype=GRB.CONTINUOUS, name="t")
s = model.addVars(days,           vtype=GRB.BINARY,    name="s")
Z = model.addVar(vtype=GRB.CONTINUOUS, name="Z")

# Objective: minimize deviation from target net calorie
model.setObjective(Z, GRB.MINIMIZE)

# Intake and burn
cal_intake = quicksum(x[i, d] * cal[i] for i in meals for d in days)
cal_burned = quicksum(t[j, d] * burn_rate[j] for j in exercises for d in days)

model.addConstr(Z >= cal_intake - cal_burned - T * target_calorie_pd)
model.addConstr(Z >= -(cal_intake - cal_burned - T * target_calorie_pd))

# Daily constraints
for d in days:
    intake_d = quicksum(x[i, d] * cal[i] for i in meals)
    burn_d = quicksum(t[j, d] * burn_rate[j] for j in exercises)
    prep_d = quicksum(x[i, d] * prep_time[i] for i in meals)
    exercise_d = quicksum(t[j, d] for j in exercises)

    # Calorie constraints based on goal
    if goal == "weight loss":
        model.addConstr(intake_d - burn_d <= target_calorie_pd)
    elif goal == "weight gain":
        model.addConstr(intake_d - burn_d >= target_calorie_pd)
    elif goal == "endurance":
        model.addConstr(intake_d - burn_d <= target_calorie_pd + 50)
        model.addConstr(intake_d - burn_d >= target_calorie_pd - 50)

    # Time constraints
    model.addConstr(prep_d <= MPT_day)
    model.addConstr(prep_d + exercise_d <= FT_day)

    # Exactly M meals
    model.addConstr(quicksum(x[i, d] for i in meals) == M)

    # --- LINKING s[d] AND y[j,d] ---
    # 1) if s[d]=1 then at least one y[j,d]=1:
    model.addConstr(
        quicksum(y[j,d] for j in exercises) >= s[d],
    )

    # 2) if s[d]=0 then y[j,d]=0 for all j:
    for j in exercises:
        model.addConstr(
            y[j,d] <= s[d],
        )

    for j in exercises:
        model.addConstr(t[j, d] <= y[j, d] * max_time[j])

    # No back‐to‐back repeats
    if d < T-1:
        for i in meals:
            model.addConstr(x[i,d] + x[i,d+1] <= 1)
        for j in exercises:
            model.addConstr(y[j,d] + y[j,d+1] <= 1)

    # Macronutrient constraints
    fat_cal = quicksum(x[i, d] * fat[i] * 9 for i in meals)
    carb_cal = quicksum(x[i, d] * carb[i] * 4 for i in meals)
    protein_cal = quicksum(x[i, d] * protein[i] * 4 for i in meals)

    if goal in {"weight loss", "endurance"}:
        model.addConstr(fat_cal >= 0.2 * target_calorie_pd)
        model.addConstr(fat_cal <= 0.35 * target_calorie_pd)
        model.addConstr(carb_cal >= 0.45 * target_calorie_pd)
        model.addConstr(carb_cal <= 0.65 * target_calorie_pd)
        model.addConstr(protein_cal >= 0.1 * target_calorie_pd)
        model.addConstr(protein_cal <= 0.35 * target_calorie_pd)
    elif goal == "weight gain":
        model.addConstr(fat_cal >= 0.15 * target_calorie_pd)
        model.addConstr(fat_cal <= 0.3 * target_calorie_pd)
        model.addConstr(carb_cal >= 0.45 * target_calorie_pd)
        model.addConstr(carb_cal <= 0.6 * target_calorie_pd)
        model.addConstr(protein_cal >= 0.3 * target_calorie_pd)
        model.addConstr(protein_cal <= 0.35 * target_calorie_pd)

# Weekly workout constraint
for w in range(num_weeks):
    week_days = range(w * 7, (w + 1) * 7)
    model.addConstr(quicksum(s[d] for d in week_days) == W)

if has_partial_week:
    last_week_days = list(range(num_weeks * 7, T))
    L = len(last_week_days)
    req = min(L, W)                            # the formulation’s min(|D_last|, W)
    model.addConstr(
        quicksum(s[d] for d in last_week_days) == req
    )
    
# Optimize
model.optimize()

# Extract solution
if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
    print(f"Solution found with objective value: {model.objVal}")
    
    today = datetime.now()
    week_net_calories = 0
    week_total_time = 0
    workout_time_week = 0
    workout_days_count = 0

    for d in days:
        current_date = today + timedelta(days=d)
        meals_d = [i for i in meals if x[i, d].x > 0.5]
        
        # Only include exercises with positive duration (t > 0)
        exercises_d = [(j, t[j, d].x) for j in exercises if t[j, d].x > 0.001]  # Using small threshold to account for floating point precision

        intake = sum(cal[i] for i in meals_d)
        burned = sum(t[j, d].x * burn_rate[j] for j, _ in exercises_d)
        net = intake - burned
        meal_prep_time = sum(prep_time[i] for i in meals_d)
        exercise_time = sum(time for _, time in exercises_d)
        total_time = meal_prep_time + exercise_time
        day_date = current_date

        week_net_calories += net
        week_total_time += total_time
        workout_time_week += exercise_time
        if len(exercises_d) > 0:
            workout_days_count += 1

        if d % 7 == 0:
            print(f"\n=== Week {(d // 7) + 1} ===")

        print(f"Day {d + 1} ({day_date.strftime('%Y-%m-%d')}):")

        print(" Selected Meals:")
        for i in meals_d:
            print(f"    - {recipe[i]}, calories:{cal[i]} cal, carb: {carb[i]}g, protein: {protein[i]}g, fat: {fat[i]}g, total time: {prep_time[i]} mins")

        print(" Selected Exercises:")
        if exercises_d:
            for j, time in exercises_d:
                print(f"    - {ex_name[j]}, type: {workout_type[j]}, location: {location[j]}, duration: {time:.1f} mins, estimated calories burned: {burn_rate[j] * time:.1f} cal")
        else:
            print("    - No exercises scheduled for this day")

        print(f"  Total Time Spent: {total_time:.1f} mins")
        print(f"  Total Net Calories: {net:.1f} kcal")

    
elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Try relaxing some constraints.")
    # Compute and print the Irreducible Inconsistent Subsystem (IIS)
    model.computeIIS()
    print("The following constraints are causing the infeasibility:")
    for c in model.getConstrs():
        if c.IISConstr:
            print(f"  {c.ConstrName}")
elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded.")
else:
    print(f"Optimization ended with status {model.status}")

Solution found with objective value: 0.3899999987260294

=== Week 1 ===
Day 1 (2025-04-20):
 Selected Meals:
    - Crispy Walnut Crumbles, calories:80.0 cal, carb: 7.0g, protein: 7.0g, fat: 3.0g, total time: 18.0 mins
    - Watercress Detox Delight, calories:150.0 cal, carb: 8.0g, protein: 7.0g, fat: 10.0g, total time: 5.0 mins
    - Bloody Marys, calories:237.7 cal, carb: 42.54g, protein: 10.36g, fat: 2.9g, total time: 21.0 mins
 Selected Exercises:
    - No exercises scheduled for this day
  Total Time Spent: 44.0 mins
  Total Net Calories: 467.7 kcal
Day 2 (2025-04-21):
 Selected Meals:
    - Balanced Alkaline Veggie Cubes, calories:20.0 cal, carb: 5.0g, protein: 11.0g, fat: 0.2g, total time: 5.0 mins
    - Refreshing Watermelon Breather, calories:55.0 cal, carb: 9.0g, protein: 0.9g, fat: 1.3g, total time: 5.0 mins
    - Tzatziki, calories:392.68 cal, carb: 40.51g, protein: 20.31g, fat: 16.6g, total time: 30.0 mins
 Selected Exercises:
    - No exercises scheduled for this day
  Tot

## Putting all of them together: Formatting the data into a format that Backend returns

In [10]:
from datetime import datetime, date, timedelta, timezone
from typing import List, Dict
from pydantic import BaseModel
from gurobipy import Model, GRB, quicksum

##############################
# Define Output Schema Models
##############################

class Meal(BaseModel):
    recipe: str
    calories: float
    macros: Dict[str, float]  # e.g. {"carbs": 55.0, "protein": 10.0, "fat": 8.0}
    total_time: float         # In minutes

class Exercise(BaseModel):
    name: str
    type: str
    location: str
    duration: float                # In minutes
    estimated_calories_burned: float

class DailyPlan(BaseModel):
    day: date
    selected_meals: List[Meal]
    selected_exercises: List[Exercise]
    total_time_used: float         # Combined time for meal prep and exercise (in minutes)
    total_net_calories: float      # Intake (from meals) minus calories burned (from exercise)

class WeeklyInfo(BaseModel):
    free_time_week: float             # Total free time available per week (in minutes)
    avg_free_time_used: float         # Average total time used per day (in minutes)
    avg_workout_duration: float       # Average workout duration per day (in minutes)
    meals_per_day: float              # Meals per day (as provided)
    avg_net_calories: float           # Average net calories per day

class OptimizationResult(BaseModel):
    plan: List[DailyPlan]
    weekly_info: WeeklyInfo
    status: str                     # The Gurobi model status (e.g., "OPTIMAL", "TIME_LIMIT", "INFEASIBLE")
    recommendations: List[str]      # List of recommendations to adjust backend inputs if needed

In [11]:
def diagnose_model(diets_df, ex_df, user_params, metrics) -> List[str]:
    """
    Aggregates smarter diagnostic recommendations for when the model is infeasible.
    Instead of listing all the computed numbers, it produces a summary
    and a concise recommendation of which input(s) might be changed.
    """
    recommendations = []

    T = metrics["days_to_target"]
    target_calorie_pd = metrics["target_calorie_per_day"]
    MPT_day = user_params.get("mealPrepTime") * user_params.get("mealsPerDay")
    FT_day = user_params["freeTime"] * 60
    M = user_params["mealsPerDay"]

    # Build a summary string of the key inputs:
    summary = (f"Planning horizon: {T} days; Target calories/day: {target_calorie_pd:.2f}; "
               f"Meal prep time: {MPT_day} min/day; Free time: {FT_day} min/day; Meals per day: {M}")

    # 1. Weight Change Feasibility:
    weight_change = user_params["goalWeight"] - user_params["weight"]
    if abs(weight_change) > 10 and T < 14:
        recommendations.append(f"Your target weight change ({weight_change} kg) is very aggressive for such a short period ({T} days). Consider reducing your weight change target or extending your planning horizon.")

    # 2. Meal Calorie Feasibility:
    min_calories = diets_df['calories'].min()
    max_calories = diets_df['calories'].max()
    min_daily_calories = min_calories * M
    max_daily_calories = max_calories * M
    # For weight loss, a low target calorie may be unachievable; for weight gain, a high target may be
    if target_calorie_pd < min_daily_calories:
        recommendations.append(f"Your targetted daily calorie intake of {target_calorie_pd} is too low for the current number of meals ({M} meals). Consider reducing the number of meals per day or extending your timeframe of {T} days.")
    elif target_calorie_pd > max_daily_calories:
        recommendations.append(f"Your targetted daily calorie intake of {target_calorie_pd} is too high for the current number of meals ({M} meals). Consider increasing the number of meals per day or extending your timeframe of {T} days.")

    # 3. Meal Prep Time Feasibility:
    min_meal_time = diets_df['total_time_in_minutes'].min()
    total_min_prep_time = min_meal_time * M
    if total_min_prep_time > MPT_day:
        recommendations.append("The fastest possible meal prep time exceeds your available time. Consider reducing meals per day or increasing your available meal prep time.")

    # 4. Macronutrient Feasibility:
    goal = user_params["goalType"]
    if goal in {"weight_loss", "endurance"}:
        fat_min, fat_max = 0.2, 0.35
        carb_min, carb_max = 0.45, 0.65
        protein_min, protein_max = 0.1, 0.35
    else:
        fat_min, fat_max = 0.15, 0.3
        carb_min, carb_max = 0.45, 0.6
        protein_min, protein_max = 0.3, 0.35

    meals_in_range = 0
    for _, meal in diets_df.iterrows():
        fat_cal = meal['fat'] * 9
        carb_cal = meal['carbs'] * 4
        protein_cal = meal['protein'] * 4
        total_cal = fat_cal + carb_cal + protein_cal
        fat_pct = fat_cal / total_cal
        carb_pct = carb_cal / total_cal
        protein_pct = protein_cal / total_cal
        if (fat_min <= fat_pct <= fat_max and
            carb_min <= carb_pct <= carb_max and
            protein_min <= protein_pct <= protein_max):
            meals_in_range += 1

    if meals_in_range < M:
        recommendations.append("There are not enough meals in our dataset that meet the desired macronutrient balance. Consider increasing the number of meals per day.")

    # 5. Exercise Options Feasibility:
    if len(ex_df) == 0:
        recommendations.append("No exercise options are available. Consider contacting support to add more exercise data.")

    # Combine the summary and the recommendations. (If no specific recommendation, just return the summary.)
    if recommendations:
        return [summary] + recommendations
    else:
        return [summary, "All input parameters appear feasible."]


In [12]:
GUROBI_STATUS_CODES = {
    1: "LOADED",
    2: "OPTIMAL",
    3: "INFEASIBLE",
    4: "INF_OR_UNBD",
    5: "UNBOUNDED",
    6: "CUTOFF",
    7: "ITERATION_LIMIT",
    8: "NODE_LIMIT",
    9: "TIME_LIMIT",
    10: "SOLUTION_LIMIT",
    11: "INTERRUPTED",
    12: "NUMERIC",
    13: "SUBOPTIMAL",
    14: "INPROGRESS",
    15: "USER_OBJ_LIMIT",
    16: "WORK_LIMIT",
    17: "MEM_LIMIT"
}

def generate_optimization_output(model: Model,
                                 days: range,
                                 meals: List,
                                 exercises: List,
                                 x, t,
                                 recipe: Dict,
                                 cal: Dict,
                                 fat: Dict,
                                 carb: Dict,
                                 protein: Dict,
                                 prep_time: Dict,
                                 ex_name: Dict,
                                 workout_type: Dict,
                                 location: Dict,
                                 burn_rate: Dict,
                                 user_params: Dict,
                                 metrics: Dict,
                                 diets_df, ex_df) -> str:
    """
    Extracts the solution from the Gurobi model and aggregates the result in a JSON-compatible
    dictionary that conforms to the output schema.
    In addition to the optimization result (plan and weekly_info), it adds a 'status' field and
    a 'recommendations' field in case of failure.
    """
    # Start date for the plan is assumed to be today's date.
    start_date = datetime.now().date()
    plan: List[DailyPlan] = []
    recommendations: List[str] = []

    # Check model status: if solved (OPTIMAL or TIME_LIMIT) then extract solution.
    if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT]:
        total_time_week = 0.0
        total_exercise_time_week = 0.0
        total_net_cal_week = 0.0
        days_count_for_week = 0

        for d in days:
            current_date = start_date + timedelta(days=d)
            # Retrieve selected meals (using a threshold on x)
            selected_meal_indices = [i for i in meals if x[i, d].x > 0.5]
            # Retrieve exercises with positive duration.
            selected_exercise_indices = [(j, t[j, d].x) for j in exercises if t[j, d].x > 0.001]

            # Calculate daily totals.
            intake = sum(cal[i] for i in selected_meal_indices)
            burned = sum(t_val * burn_rate[j] for j, t_val in selected_exercise_indices)
            net = intake - burned
            meal_time_total = sum(prep_time[i] for i in selected_meal_indices)
            exercise_time_total = sum(t_val for _, t_val in selected_exercise_indices)
            total_time_used = meal_time_total + exercise_time_total

            # Construct Meal objects.
            selected_meals = []
            for i in selected_meal_indices:
                macros = {"carbs": float(carb[i]), "protein": float(protein[i]), "fat": float(fat[i])}
                meal_obj = Meal(
                    recipe=recipe[i],
                    calories=float(cal[i]),
                    macros=macros,
                    total_time=float(prep_time[i])
                )
                selected_meals.append(meal_obj)

            # Construct Exercise objects.
            selected_exercises = []
            for j, duration in selected_exercise_indices:
                ex_obj = Exercise(
                    name=ex_name[j],
                    type=workout_type[j],
                    location=location[j],
                    duration=duration,
                    estimated_calories_burned=duration * burn_rate[j]
                )
                selected_exercises.append(ex_obj)

            daily_plan = DailyPlan(
                day=current_date,
                selected_meals=selected_meals,
                selected_exercises=selected_exercises,
                total_time_used=total_time_used,
                total_net_calories=net
            )
            plan.append(daily_plan)

            if d < 7:  # For weekly aggregates, use the first 7 days.
                total_time_week += total_time_used
                total_exercise_time_week += exercise_time_total
                total_net_cal_week += net
                days_count_for_week += 1

        if days_count_for_week > 0:
            avg_free_time_used = total_time_week / days_count_for_week
            avg_workout_duration = total_exercise_time_week / days_count_for_week
            avg_net_calories = total_net_cal_week / days_count_for_week
        else:
            avg_free_time_used = avg_workout_duration = avg_net_calories = 0.0

        free_time_week = user_params.get("freeTime", 0) * 60 * user_params.get("daysWeek", 7)

        weekly_info = WeeklyInfo(
            free_time_week=free_time_week,
            avg_free_time_used=avg_free_time_used,
            avg_workout_duration=avg_workout_duration,
            meals_per_day=user_params.get("mealsPerDay", 0),
            avg_net_calories=avg_net_calories
        )
        status_str = GUROBI_STATUS_CODES[model.status]

    # If the model is infeasible, run diagnostics and gather recommendations.
    elif model.status == GRB.INFEASIBLE:
        recommendations = diagnose_model(diets_df, ex_df, user_params, metrics)
        status_str = GUROBI_STATUS_CODES[model.status]
        weekly_info = WeeklyInfo(
            free_time_week=0,
            avg_free_time_used=0,
            avg_workout_duration=0,
            meals_per_day=user_params.get("mealsPerDay", 0),
            avg_net_calories=0
        )
        plan = []  # No daily plans available.

    else:
        recommendations = [f"Model ended with status {model.status}."]
        status_str = GUROBI_STATUS_CODES[model.status]
        weekly_info = WeeklyInfo(
            free_time_week=0,
            avg_free_time_used=0,
            avg_workout_duration=0,
            meals_per_day=user_params.get("mealsPerDay", 0),
            avg_net_calories=0
        )
        plan = []

    # Assemble the final output dictionary.
    output = OptimizationResult(
        plan=plan,
        weekly_info=weekly_info,
        status=status_str,
        recommendations=recommendations
    )
    import json
    return json.dumps(output, indent=2, default=str)

## Optimal Case, No Recommendations to give to frontend

- Sufficient time to lose a target weight of 5kg with sufficient free time

In [13]:
result = generate_optimization_output(
    model=model,
    days=range(metrics["days_to_target"]),
    meals=list(diets_df.index),
    exercises=list(ex_df.index),
    x=x,
    t=t,
    recipe={i: diets_df.loc[i, 'recipe'] for i in diets_df.index},
    cal={i: diets_df.loc[i, 'calories'] for i in diets_df.index},
    fat={i: diets_df.loc[i, 'fat'] for i in diets_df.index},
    carb={i: diets_df.loc[i, 'carbs'] for i in diets_df.index},
    protein={i: diets_df.loc[i, 'protein'] for i in diets_df.index},
    prep_time={i: diets_df.loc[i, 'total_time_in_minutes'] for i in diets_df.index},
    ex_name={j: ex_df.loc[j, 'exercise_name'] for j in ex_df.index},
    workout_type={j: ex_df.loc[j, 'activity_type'] for j in ex_df.index},
    location={j: ex_df.loc[j, 'workout_location'] for j in ex_df.index},
    burn_rate={j: ex_df.loc[j, 'calories_burned_per_min'] for j in ex_df.index},
    user_params=sample_user,  # Your sample request body
    metrics=metrics,
    diets_df=diets_df,
    ex_df=ex_df
)

# Then print or return the JSON result to the frontend:
print(result)

"plan=[DailyPlan(day=datetime.date(2025, 4, 20), selected_meals=[Meal(recipe='Garlic Touched Beans', calories=215.0, macros={'carbs': 7.0, 'protein': 4.0, 'fat': 9.0}, total_time=10.0), Meal(recipe='Hearty Cucumber Quencher', calories=180.0, macros={'carbs': 18.0, 'protein': 5.0, 'fat': 2.0}, total_time=5.0), Meal(recipe='Salsa', calories=280.39, macros={'carbs': 50.01, 'protein': 10.75, 'fat': 4.15}, total_time=24.0)], selected_exercises=[Exercise(name='Seated Rows', type='General', location='gym', duration=14.803125000000003, estimated_calories_burned=118.42500000000003), Exercise(name='Handball', type='Sports', location='sports centre', duration=6.196874999999996, estimated_calories_burned=89.23499999999994)], total_time_used=60.0, total_net_calories=467.73), DailyPlan(day=datetime.date(2025, 4, 21), selected_meals=[Meal(recipe='Walnuts And Asparagus Delight', calories=124.0, macros={'carbs': 2.0, 'protein': 3.0, 'fat': 12.0}, total_time=10.0), Meal(recipe='The Ultimate Morning Gree

## Testing of more cases

### Helper function to make our lives easier to avoid dupication of code

In [22]:
def solve_optimization(user_params: dict) -> dict:
    """
    Runs the full optimization process given user parameters.
    It filters the data, computes metrics, builds and optimizes the model,
    and returns the results as a dictionary conforming to the output schema.
    """
    # --- Preprocessing ---
    filtered_diets = filter_diets(user_params)
    filtered_exercises = filter_exercises(user_params)
    metrics = compute_user_metrics(user_params)

    # Setup parameters
    T = metrics["days_to_target"]
    target_calorie_pd = metrics["target_calorie_per_day"]
    goal = user_params["goalType"].replace("_", " ")
    FT_day = user_params["freeTime"] * 60
    MPT_day = user_params["mealPrepTime"] * user_params["mealsPerDay"]
    M = user_params["mealsPerDay"]
    W = user_params["daysWeek"]
    num_weeks = T // 7
    has_partial_week = T % 7 > 0

    # --- Build Model ---
    model = Model("FitPlanner")
    model.Params.OutputFlag = 0
    model.setParam('TimeLimit', 300)

    days = range(T)
    meals_idx = list(filtered_diets.index)
    exercises_idx = list(filtered_exercises.index)

    # Extract parameters
    recipe     = {i: filtered_diets.loc[i, 'recipe'] for i in meals_idx}
    cal        = {i: filtered_diets.loc[i, 'calories'] for i in meals_idx}
    fat        = {i: filtered_diets.loc[i, 'fat'] for i in meals_idx}
    carb       = {i: filtered_diets.loc[i, 'carbs'] for i in meals_idx}
    protein    = {i: filtered_diets.loc[i, 'protein'] for i in meals_idx}
    prep_time  = {i: filtered_diets.loc[i, 'total_time_in_minutes'] for i in meals_idx}
    ex_name    = {j: filtered_exercises.loc[j, 'exercise_name'] for j in exercises_idx}
    location   = {j: filtered_exercises.loc[j, 'workout_location'] for j in exercises_idx}
    burn_rate  = {j: filtered_exercises.loc[j, 'calories_burned_per_min'] for j in exercises_idx}
    workout_type = {j: filtered_exercises.loc[j, 'activity_type'] for j in exercises_idx}

    max_time = {
        j: 60  if workout_type[j]=="Sports"
           else 30  if workout_type[j]=="Outdoor/Water"
           else 20
        for j in exercises_idx
    }

    # Decision variables
    x     = model.addVars(meals_idx, days,     vtype=GRB.BINARY,    name="x")
    y     = model.addVars(exercises_idx, days, vtype=GRB.BINARY,    name="y")
    t_var = model.addVars(exercises_idx, days, vtype=GRB.CONTINUOUS, name="t")
    s     = model.addVars(days,             vtype=GRB.BINARY,    name="s")
    Z     = model.addVar(vtype=GRB.CONTINUOUS, name="Z")

    # Objective
    model.setObjective(Z, GRB.MINIMIZE)
    total_intake = quicksum(x[i,d]*cal[i]       for i in meals_idx for d in days)
    total_burn   = quicksum(t_var[j,d]*burn_rate[j] for j in exercises_idx for d in days)
    model.addConstr(Z >= total_intake - total_burn - T * target_calorie_pd)
    model.addConstr(Z >= -(total_intake - total_burn - T * target_calorie_pd))

    # Per‐day constraints
    for d in days:
        intake_d   = quicksum(x[i, d]*cal[i]        for i in meals_idx)
        burn_d     = quicksum(t_var[j, d]*burn_rate[j] for j in exercises_idx)
        prep_d     = quicksum(x[i, d]*prep_time[i]  for i in meals_idx)
        exercise_d = quicksum(t_var[j, d]           for j in exercises_idx)

        # Calorie balance
        if goal == "weight loss":
            model.addConstr(intake_d - burn_d <= target_calorie_pd)
        elif goal == "weight gain":
            model.addConstr(intake_d - burn_d >= target_calorie_pd)
        else:  # endurance
            model.addConstr(intake_d - burn_d <= target_calorie_pd + 50)
            model.addConstr(intake_d - burn_d >= target_calorie_pd - 50)

        # Time
        model.addConstr(prep_d <= MPT_day)
        model.addConstr(prep_d + exercise_d <= FT_day)

        # Meals count
        model.addConstr(quicksum(x[i, d] for i in meals_idx) == M)

        # --- LINK s[d] WITH y[j,d] ---
        # 1) if s[d]=1 then at least one y[j,d]=1:
        model.addConstr(
            quicksum(y[j, d] for j in exercises_idx) >= s[d],
            name=f"flag_lower_{d}"
        )  # <<< CHANGED >>>

        # 2) if s[d]=0 then y[j,d]=0 for all j:
        for j in exercises_idx:
            model.addConstr(
                y[j, d] <= s[d],
                name=f"flag_upper_{j}_{d}"
            )  # <<< CHANGED >>>

        # Max durations
        for j in exercises_idx:
            model.addConstr(t_var[j, d] <= y[j, d] * max_time[j])

        # No repeats
        if d < T - 1:
            for i in meals_idx:
                model.addConstr(x[i, d] + x[i, d+1] <= 1)
            for j in exercises_idx:
                model.addConstr(y[j, d] + y[j, d+1] <= 1)

        # Macronutrients
        fat_cal     = quicksum(x[i, d]*fat[i]*9      for i in meals_idx)
        carb_cal    = quicksum(x[i, d]*carb[i]*4     for i in meals_idx)
        protein_cal = quicksum(x[i, d]*protein[i]*4  for i in meals_idx)

        if goal in {"weight loss", "endurance"}:
            model.addConstr(fat_cal     >= 0.20 * target_calorie_pd)
            model.addConstr(fat_cal     <= 0.35 * target_calorie_pd)
            model.addConstr(carb_cal    >= 0.45 * target_calorie_pd)
            model.addConstr(carb_cal    <= 0.65 * target_calorie_pd)
            model.addConstr(protein_cal >= 0.10 * target_calorie_pd)
            model.addConstr(protein_cal <= 0.35 * target_calorie_pd)
        else:  # weight gain
            model.addConstr(fat_cal     >= 0.15 * target_calorie_pd)
            model.addConstr(fat_cal     <= 0.30 * target_calorie_pd)
            model.addConstr(carb_cal    >= 0.45 * target_calorie_pd)
            model.addConstr(carb_cal    <= 0.60 * target_calorie_pd)
            model.addConstr(protein_cal >= 0.30 * target_calorie_pd)
            model.addConstr(protein_cal <= 0.35 * target_calorie_pd)

    # Weekly workout frequency
    for w in range(num_weeks):
        week_days = range(w * 7, (w + 1) * 7)
        model.addConstr(
            quicksum(s[d] for d in week_days) == W
        )

    # Partial final week → equality with min(|D_last|, W)
    if has_partial_week:
        last_week_days = list(range(num_weeks * 7, T))  # <<< CHANGED >>>
        req = min(len(last_week_days), W)                # <<< CHANGED >>>
        model.addConstr(
            quicksum(s[d] for d in last_week_days) == req,
            name="partial_week"                          # <<< CHANGED >>>
        )

    # Solve
    model.optimize()

    # Build output
    output = generate_optimization_output(
        model=model,
        days=days,
        meals=meals_idx,
        exercises=exercises_idx,
        x=x,
        t=t_var,
        recipe=recipe,
        cal=cal,
        fat=fat,
        carb=carb,
        protein=protein,
        prep_time=prep_time,
        ex_name=ex_name,
        workout_type=workout_type,
        location=location,
        burn_rate=burn_rate,
        user_params=user_params,
        metrics=metrics,
        diets_df=filtered_diets,
        ex_df=filtered_exercises
    )
    return output

In [23]:
# A helper to run a test case.
def test_case(case_name: str, user_params: dict):
    print(f"\n===== Test Case: {case_name} =====")
    result = solve_optimization(user_params)
    print(result)

### Unrealistic weight loss

In [24]:
# Define some test scenarios:
# 1. Unrealistic weight loss: very high target weight change in too short period.
test_user_weight_loss = {
    "age": 30,
    "daysWeek": 3,
    "dietRestrictions": ["none"],
    "fitnessLevel": "beginner",
    "freeTime": 3,          # freeTime in hours per day
    "gender": "male",
    "goalTargetDate": "2025-05-01T04:48:00.000Z",  # Only a couple of days runway
    "goalType": "weight_loss",
    "goalWeight": 65,       # 20 kg weight loss
    "height": 175,
    "mealPrepTime": 15,
    "mealsPerDay": 3,
    "name": "TestUser1",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 70
}

# Run all test cases.
print(test_case("Unrealistic Weight Loss", test_user_weight_loss))


===== Test Case: Unrealistic Weight Loss =====
Filtered diets (8293 rows)
Filtered exercises (15 rows)
"plan=[] weekly_info=WeeklyInfo(free_time_week=0.0, avg_free_time_used=0.0, avg_workout_duration=0.0, meals_per_day=3.0, avg_net_calories=0.0) status='INFEASIBLE' recommendations=['Planning horizon: 12 days; Target calories/day: -1229.83; Meal prep time: 45 min/day; Free time: 180 min/day; Meals per day: 3', 'Your targetted daily calorie intake of -1229.83 is too low for the current number of meals (3 meals). Consider reducing the number of meals per day or extending your timeframe of 12 days.']"
None


### Unrealistic Weight gain


In [25]:
# 2. Unrealistic weight gain: not enough time for the desired increase.
test_user_weight_gain = {
    "age": 40,
    "daysWeek": 4,
    "dietRestrictions": ["none"],
    "fitnessLevel": "intermediate",
    "freeTime": 4,
    "gender": "female",
    "goalTargetDate": "2025-04-16T04:48:00.000Z",  # very short runway
    "goalType": "weight_gain",
    "goalWeight": 85,       # 20 kg gain
    "height": 165,
    "mealPrepTime": 20,
    "mealsPerDay": 4,
    "name": "TestUser2",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 65
}

test_case("Unrealistic Weight Gain", test_user_weight_gain)


===== Test Case: Unrealistic Weight Gain =====
Filtered diets (8293 rows)
Filtered exercises (57 rows)
"plan=[] weekly_info=WeeklyInfo(free_time_week=0.0, avg_free_time_used=0.0, avg_workout_duration=0.0, meals_per_day=4.0, avg_net_calories=0.0) status='INFEASIBLE' recommendations=['Planning horizon: 1 days; Target calories/day: 155584.30; Meal prep time: 80 min/day; Free time: 240 min/day; Meals per day: 4', 'Your target weight change (20 kg) is very aggressive for such a short period (1 days). Consider reducing your weight change target or extending your planning horizon.', 'Your targetted daily calorie intake of 155584.3 is too high for the current number of meals (4 meals). Consider increasing the number of meals per day or extending your timeframe of 1 days.']"


### Too little meals

In [26]:
# 3. Too few meals: eat 1 meal but want to gain weight
test_user_few_meals = {
    "age": 28,
    "daysWeek": 3,
    "dietRestrictions": ["none"],
    "fitnessLevel": "beginner",
    "freeTime": 5,
    "gender": "female",
    "goalTargetDate": "2025-05-01T04:48:00.000Z",
    "goalType": "weight_gain",
    "goalWeight": 70,
    "height": 160,
    "mealPrepTime": 10,
    "mealsPerDay": 1,  # Requesting 1 meals per day (likely more than available)
    "name": "TestUser3",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 65
}

test_case("Too Few Meals Available", test_user_few_meals)


===== Test Case: Too Few Meals Available =====
Filtered diets (8293 rows)
Filtered exercises (15 rows)
"plan=[] weekly_info=WeeklyInfo(free_time_week=0.0, avg_free_time_used=0.0, avg_workout_duration=0.0, meals_per_day=1.0, avg_net_calories=0.0) status='INFEASIBLE' recommendations=['Planning horizon: 12 days; Target calories/day: 4827.13; Meal prep time: 10 min/day; Free time: 300 min/day; Meals per day: 1', 'Your targetted daily calorie intake of 4827.13 is too high for the current number of meals (1 meals). Consider increasing the number of meals per day or extending your timeframe of 12 days.']"


### Too many meals requested

In [27]:
# 4. Too many meals: want to lose weight but also  eat a lot
test_user_many_meals = {
    "age": 35,
    "daysWeek": 4,
    "dietRestrictions": ["none"],
    "fitnessLevel": "advanced",
    "freeTime": 2,
    "gender": "male",
    "goalTargetDate": "2025-05-01T04:48:00.000Z",
    "goalType": "weight_loss",
    "goalWeight": 65,
    "height": 180,
    "mealPrepTime": 10,
    "mealsPerDay": 7,  # Requesting 7 meals per day (might be too many if meal prep times add up)
    "name": "TestUser4",
    "preferredLocation": "none",
    "preferredWorkoutType": "none",
    "varietyPreferences": ["none"],
    "weight": 75
}

test_case("Too Many Meals Requested", test_user_many_meals)


===== Test Case: Too Many Meals Requested =====
Filtered diets (8293 rows)
Filtered exercises (21 rows)
"plan=[] weekly_info=WeeklyInfo(free_time_week=0.0, avg_free_time_used=0.0, avg_workout_duration=0.0, meals_per_day=7.0, avg_net_calories=0.0) status='INFEASIBLE' recommendations=['Planning horizon: 12 days; Target calories/day: -4370.67; Meal prep time: 70 min/day; Free time: 120 min/day; Meals per day: 7', 'Your targetted daily calorie intake of -4370.67 is too low for the current number of meals (7 meals). Consider reducing the number of meals per day or extending your timeframe of 12 days.']"
